In [19]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import json, config #標準のjsonモジュールとconfig.pyの読み込み
from requests_oauthlib import OAuth1Session
import pandas as pd
import sys
import MeCab

#config.pyにAPIキーを書いておく。
CK = config.CONSUMER_KEY
CS = config.CONSUMER_SECRET
AT = config.ACCESS_TOKEN
ATS = config.ACCESS_TOKEN_SECRET
twitter = OAuth1Session(CK, CS, AT, ATS) #認証処理

url = "https://api.twitter.com/1.1/statuses/user_timeline.json"

#調べる対象
print("何について調べますか？→「疲れ」「飲み過ぎ」「悩み」")
target = input(">>")
if target == "疲れ":
    keyword = "疲"
elif target == "飲み過ぎ":
    keyword = "ビール"
elif target == "悩み":
    keyword = "つらい"
else:
    print("「疲れ」「飲み過ぎ」「悩み」の中から選んでください")


# とくにパラメータは無い
params = {}

# OAuth で GET
twitter = OAuth1Session(CK, CS, AT, ATS)
res = twitter.get(url, params = params)

username_column = []
text_colmn = []
date_column = []

if res.status_code == 200: #正常通信出来た場合
    timelines = json.loads(res.text) #レスポンスからタイムラインリストを取得
    for line in timelines: #タイムラインリストをループ処理
        username_column.append(line['user']['name'])
        text_colmn.append(line['text'])
        date_column.append(line['created_at'])
else: #正常通信出来なかった場合
    print("Failed: %d" % res.status_code)

df = pd.DataFrame({"user_name":username_column,"tweet":text_colmn,"date":date_column}) #後で利用しやすいようにdataframeにする
print("最近のツイートを表示します。")
print(df)

m = MeCab.Tagger("-Owakati")
keitaiso_list = []
for text in df.loc[:,["tweet"]].values.tolist():
    keitaiso_list.extend(m.parse(text[0]).split(" "))

number = 0
for keitaiso in keitaiso_list:
    if keyword in keitaiso:
        number += 1
    else:
        number = number

print("あなたの疲れ指数は:")
print(number)


何について調べますか？→「疲れ」「飲み過ぎ」「悩み」
>>疲れ
最近のツイートを表示します。
                              date  \
0   Wed Jul 04 04:03:59 +0000 2018   
1   Wed Jul 04 03:41:56 +0000 2018   
2   Wed Jul 04 03:40:20 +0000 2018   
3   Wed Jul 04 03:38:58 +0000 2018   
4   Wed Jul 04 02:45:30 +0000 2018   
5   Wed Jul 04 02:45:17 +0000 2018   
6   Wed Jul 04 02:44:45 +0000 2018   
7   Wed Jul 04 02:44:23 +0000 2018   
8   Wed Jul 04 02:44:18 +0000 2018   
9   Wed Jul 04 02:44:09 +0000 2018   
10  Tue Jul 03 06:54:45 +0000 2018   
11  Tue Jul 03 06:54:24 +0000 2018   
12  Tue Jul 03 06:53:22 +0000 2018   
13  Wed Jun 20 07:18:55 +0000 2018   

                                                tweet user_name  
0                                           疲れた疲れた疲れた   K. OTSU  
1   Github: https://t.co/XyvA90Vzbt\nQiita: https:...   K. OTSU  
2   RT @windorch_spark: 【Wind Orchestra Spark 第一回定...   K. OTSU  
3   https://t.co/HzmLGGlAVu\nMecabとTwitterAPIをとりあえ...   K. OTSU  
4                                              眠気が・・